In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK: Submit a HyperParameter tuning training job with TensorFlow


## Installation


Install the Google *cloud-storage* library as well.


In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see [Region support for Vertex services]()


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex SDK

Import the Vertex SDK into our Python environment.


In [ ]:
import json
import os
import sys
import time
from datetime import datetime

from googleapiclient import discovery

#### Vertex constants

Setup up the following constants for Vertex:

- `PARENT`: The Vertex location root path for dataset, model and endpoint resources.


In [ ]:
# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

## Clients

The Vertex SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (Vertex).

You will use several clients in this tutorial, so set them all up upfront.


In [ ]:
client = discovery.build("ml", "v1")

## Prepare a trainer script

### Package assembly

In [ ]:
# Make folder for python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\
tag_build =\n\
tag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\
# Requires TensorFlow Datasets\n\
setuptools.setup(\n\
    install_requires=[\n\
        'tensorflow_datasets==1.3.0',\n\
    ],\n\
    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\
Name: Hyperparameter Tuning - Boston Housing\n\
Version: 0.0.0\n\
Summary: Demonstration hyperparameter tuning script\n\
Home-page: www.google.com\n\
Author: Google\n\
Author-email: aferlitsch@gmail.com\n\
License: Public\n\
Description: Demo\n\
Platform: Vertex AI"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### Task.py contents

In [ ]:
%%writefile custom/trainer/task.py
# Custom Training for Boston Housing
  
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default='/tmp/saved_model', type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--units', dest='units',
                    default=64, type=int,
                    help='Number of units.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

def make_dataset():
  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)
    
  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)

# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(args.units, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(args.units, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

model = build_and_compile_dnn_model()

# Instantiate the HyperTune reporting object
hpt = HyperTune()

# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_loss',
        metric_value=logs['val_loss'],
        global_step=epoch)

# Train the model
BATCH_SIZE = 16
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=BATCH_SIZE, validation_split=0.1, callbacks=[HPTCallback()])
model.save(args.model_dir)


### Store training script on your Cloud Storage bucket

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz gs://$BUCKET_NAME/hpt_boston_housing.tar.gz

## Train a model

### [projects.jobs.create](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/projects.jobs/create)

#### Request

In [ ]:
JOB_NAME = "hyperparameter_tuning_" + TIMESTAMP

training_input = {
    "scaleTier": "CUSTOM",
    "masterType": "n1-standard-4",
    "packageUris": ["gs://" + BUCKET_NAME + "/hpt_boston_housing.tar.gz"],
    "pythonModule": "trainer.task",
    "args": [
        "--model-dir=" + "gs://{}/{}".format(BUCKET_NAME, JOB_NAME),
    ],
    "region": REGION,
    "runtimeVersion": "2.1",
    "pythonVersion": "3.7",
}

# Add hyperparameter tuning to the job config.
hyperparams = {
    "goal": "MINIMIZE",
    "hyperparameterMetricTag": "val_loss",
    "maxTrials": 6,
    "maxParallelTrials": 1,
    "params": [],
    "algorithm": "RANDOM_SEARCH",
}

hyperparams["params"].append(
    {
        "parameterName": "lr",
        "type": "DISCRETE",
        "discreteValues": [0.001, 0.01, 0.1],
        "scaleType": "UNIT_LINEAR_SCALE",
    }
)

hyperparams["params"].append(
    {
        "parameterName": "units",
        "type": "INTEGER",
        "minValue": 32,
        "maxValue": 256,
        "scaleType": "UNIT_LINEAR_SCALE",
    }
)

# Add hyperparameter specification to the training inputs dictionary.
training_input["hyperparameters"] = hyperparams

body = {"jobId": JOB_NAME, "trainingInput": training_input}

request = (
    client.projects()
    .jobs()
    .create(
        parent="projects/" + PROJECT_ID,
    )
)
request.body = body

print(json.dumps(json.loads(request.to_json()), indent=2))

request = client.projects().jobs().create(parent="projects/" + PROJECT_ID, body=body)

*Example output*:
```
{
  "uri": "https://ml.googleapis.com/v1/projects/migration-ucaip-training/jobs?alt=json",
  "method": "POST",
  "body": {
    "jobId": "hyperparameter_tuning_20210226020553",
    "trainingInput": {
      "scaleTier": "CUSTOM",
      "masterType": "n1-standard-4",
      "packageUris": [
        "gs://migration-ucaip-trainingaip-20210226020553/hpt_boston_housing.tar.gz"
      ],
      "pythonModule": "trainer.task",
      "args": [
        "--model-dir=gs://migration-ucaip-trainingaip-20210226020553/hyperparameter_tuning_20210226020553"
      ],
      "region": "us-central1",
      "runtimeVersion": "2.1",
      "pythonVersion": "3.7",
      "hyperparameters": {
        "goal": "MINIMIZE",
        "hyperparameterMetricTag": "val_loss",
        "maxTrials": 6,
        "maxParallelTrials": 1,
        "params": [
          {
            "parameterName": "lr",
            "type": "DISCRETE",
            "discreteValues": [
              0.001,
              0.01,
              0.1
            ],
            "scaleType": "UNIT_LINEAR_SCALE"
          },
          {
            "parameterName": "units",
            "type": "INTEGER",
            "minValue": 32,
            "maxValue": 256,
            "scaleType": "UNIT_LINEAR_SCALE"
          }
        ],
        "algorithm": "RANDOM_SEARCH"
      }
    }
  },
  "headers": {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "(gzip)",
    "x-goog-api-client": "gdcl/1.12.8 gl-python/3.7.8"
  },
  "methodId": "ml.projects.jobs.create",
  "resumable": null,
  "response_callbacks": [],
  "_in_error_state": false,
  "body_size": 0,
  "resumable_uri": null,
  "resumable_progress": 0
}
```


#### Call

In [ ]:
result = request.execute()

#### Response

In [ ]:
print(json.dumps(result, indent=2))

*Example output*:
```
{
  "jobId": "hyperparameter_tuning_20210226020553",
  "trainingInput": {
    "scaleTier": "CUSTOM",
    "masterType": "n1-standard-4",
    "packageUris": [
      "gs://migration-ucaip-trainingaip-20210226020553/hpt_boston_housing.tar.gz"
    ],
    "pythonModule": "trainer.task",
    "args": [
      "--model-dir=gs://migration-ucaip-trainingaip-20210226020553/hyperparameter_tuning_20210226020553"
    ],
    "hyperparameters": {
      "goal": "MINIMIZE",
      "params": [
        {
          "parameterName": "lr",
          "type": "DISCRETE",
          "discreteValues": [
            0.001,
            0.01,
            0.1
          ],
          "scaleType": "UNIT_LINEAR_SCALE"
        },
        {
          "parameterName": "units",
          "minValue": 32,
          "maxValue": 256,
          "type": "INTEGER",
          "scaleType": "UNIT_LINEAR_SCALE"
        }
      ],
      "maxTrials": 6,
      "maxParallelTrials": 1,
      "hyperparameterMetricTag": "val_loss",
      "algorithm": "RANDOM_SEARCH"
    },
    "region": "us-central1",
    "runtimeVersion": "2.1",
    "pythonVersion": "3.7"
  },
  "createTime": "2021-02-26T02:06:01Z",
  "state": "QUEUED",
  "trainingOutput": {
    "isHyperparameterTuningJob": true
  },
  "etag": "MPeWDTbQOUQ="
}
```


In [ ]:
# The short numeric ID for the custom training job
hyperparameter_tuning_short_id = result["jobId"]
# The full unique ID for the custom training job
hyperparameter_tuning_id = "projects/" + PROJECT_ID + "/jobs/" + result["jobId"]

print(hyperparameter_tuning_id)

### [projects.jobs.get](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/projects.jobs/get)

#### Call

In [ ]:
request = client.projects().jobs().get(name=hyperparameter_tuning_id)

result = request.execute()

#### Response

In [ ]:
print(json.dumps(result, indent=2))

*Example output*:
```
{
  "jobId": "hyperparameter_tuning_20210226020553",
  "trainingInput": {
    "scaleTier": "CUSTOM",
    "masterType": "n1-standard-4",
    "packageUris": [
      "gs://migration-ucaip-trainingaip-20210226020553/hpt_boston_housing.tar.gz"
    ],
    "pythonModule": "trainer.task",
    "args": [
      "--model-dir=gs://migration-ucaip-trainingaip-20210226020553/hyperparameter_tuning_20210226020553"
    ],
    "hyperparameters": {
      "goal": "MINIMIZE",
      "params": [
        {
          "parameterName": "lr",
          "type": "DISCRETE",
          "discreteValues": [
            0.001,
            0.01,
            0.1
          ],
          "scaleType": "UNIT_LINEAR_SCALE"
        },
        {
          "parameterName": "units",
          "minValue": 32,
          "maxValue": 256,
          "type": "INTEGER",
          "scaleType": "UNIT_LINEAR_SCALE"
        }
      ],
      "maxTrials": 6,
      "maxParallelTrials": 1,
      "hyperparameterMetricTag": "val_loss",
      "algorithm": "RANDOM_SEARCH"
    },
    "region": "us-central1",
    "runtimeVersion": "2.1",
    "pythonVersion": "3.7"
  },
  "createTime": "2021-02-26T02:06:01Z",
  "state": "QUEUED",
  "trainingOutput": {
    "isHyperparameterTuningJob": true
  },
  "etag": "GNEDjq+ds8I="
}
```


## Wait for the study to complete

In [ ]:
while True:
    response = client.projects().jobs().get(name=hyperparameter_tuning_id).execute()

    if response["state"] != "SUCCEEDED":
        print("Study trials have not completed:", response["state"])
        if response["state"] == "FAILED":
            break
    else:
        print("Study trials have completed:")
        break
    time.sleep(60)

In [ ]:
print(json.dumps(response, indent=2))

*Example output*:
```
{
  "jobId": "hyperparameter_tuning_20210226020553",
  "trainingInput": {
    "scaleTier": "CUSTOM",
    "masterType": "n1-standard-4",
    "packageUris": [
      "gs://migration-ucaip-trainingaip-20210226020553/hpt_boston_housing.tar.gz"
    ],
    "pythonModule": "trainer.task",
    "args": [
      "--model-dir=gs://migration-ucaip-trainingaip-20210226020553/hyperparameter_tuning_20210226020553"
    ],
    "hyperparameters": {
      "goal": "MINIMIZE",
      "params": [
        {
          "parameterName": "lr",
          "type": "DISCRETE",
          "discreteValues": [
            0.001,
            0.01,
            0.1
          ],
          "scaleType": "UNIT_LINEAR_SCALE"
        },
        {
          "parameterName": "units",
          "minValue": 32,
          "maxValue": 256,
          "type": "INTEGER",
          "scaleType": "UNIT_LINEAR_SCALE"
        }
      ],
      "maxTrials": 6,
      "maxParallelTrials": 1,
      "hyperparameterMetricTag": "val_loss",
      "algorithm": "RANDOM_SEARCH"
    },
    "region": "us-central1",
    "runtimeVersion": "2.1",
    "pythonVersion": "3.7"
  },
  "createTime": "2021-02-26T02:06:01Z",
  "startTime": "2021-02-26T02:06:03Z",
  "endTime": "2021-02-26T02:35:25Z",
  "state": "SUCCEEDED",
  "trainingOutput": {
    "completedTrialCount": "6",
    "trials": [
      {
        "trialId": "3",
        "hyperparameters": {
          "lr": "0.1",
          "units": "39"
        },
        "finalMetric": {
          "trainingStep": "8",
          "objectiveValue": 45.76672372585389
        },
        "startTime": "2021-02-26T02:16:50.943369954Z",
        "endTime": "2021-02-26T02:20:15Z",
        "state": "SUCCEEDED"
      },
      {
        "trialId": "5",
        "hyperparameters": {
          "lr": "0.1",
          "units": "169"
        },
        "finalMetric": {
          "trainingStep": "19",
          "objectiveValue": 46.09460951642292
        },
        "startTime": "2021-02-26T02:26:23.876101420Z",
        "endTime": "2021-02-26T02:29:59Z",
        "state": "SUCCEEDED"
      },
      {
        "trialId": "2",
        "hyperparameters": {
          "lr": "0.1",
          "units": "69"
        },
        "finalMetric": {
          "trainingStep": "19",
          "objectiveValue": 51.7642993461795
        },
        "startTime": "2021-02-26T02:12:03.812631906Z",
        "endTime": "2021-02-26T02:15:30Z",
        "state": "SUCCEEDED"
      },
      {
        "trialId": "1",
        "hyperparameters": {
          "lr": "0.01",
          "units": "232"
        },
        "finalMetric": {
          "trainingStep": "19",
          "objectiveValue": 55.58717541578339
        },
        "startTime": "2021-02-26T02:06:40.790962827Z",
        "endTime": "2021-02-26T02:10:47Z",
        "state": "SUCCEEDED"
      },
      {
        "trialId": "6",
        "hyperparameters": {
          "lr": "0.001",
          "units": "123"
        },
        "finalMetric": {
          "trainingStep": "19",
          "objectiveValue": 76.20946111911681
        },
        "startTime": "2021-02-26T02:31:11.163541810Z",
        "endTime": "2021-02-26T02:34:36Z",
        "state": "SUCCEEDED"
      },
      {
        "trialId": "4",
        "hyperparameters": {
          "lr": "0.01",
          "units": "246"
        },
        "finalMetric": {
          "trainingStep": "11",
          "objectiveValue": 100.75884600383479
        },
        "startTime": "2021-02-26T02:21:36.635969861Z",
        "endTime": "2021-02-26T02:25:02Z",
        "state": "SUCCEEDED"
      }
    ],
    "consumedMLUnits": 0.39,
    "isHyperparameterTuningJob": true,
    "hyperparameterMetricTag": "val_loss"
  },
  "etag": "eUgnylIe/+0="
}
```


## Review the results of the study

In [ ]:
best = (None, None, None, 0.0)
response = client.projects().jobs().get(name=hyperparameter_tuning_id).execute()
for trial in response["trainingOutput"]["trials"]:
    print(json.dumps(trial, indent=2))
    # Keep track of the best outcome
    try:
        if float(trial.final_measurement.metrics[0].value) > best[3]:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
    except:
        pass

print()
print("ID", best[0])
print("Decay", best[1])
print("Learning Rate", best[2])
print("Validation Accuracy", best[3])

*Example output*:
```
{
  "trialId": "3",
  "hyperparameters": {
    "lr": "0.1",
    "units": "39"
  },
  "finalMetric": {
    "trainingStep": "8",
    "objectiveValue": 45.76672372585389
  },
  "startTime": "2021-02-26T02:16:50.943369954Z",
  "endTime": "2021-02-26T02:20:15Z",
  "state": "SUCCEEDED"
}
{
  "trialId": "5",
  "hyperparameters": {
    "lr": "0.1",
    "units": "169"
  },
  "finalMetric": {
    "trainingStep": "19",
    "objectiveValue": 46.09460951642292
  },
  "startTime": "2021-02-26T02:26:23.876101420Z",
  "endTime": "2021-02-26T02:29:59Z",
  "state": "SUCCEEDED"
}
{
  "trialId": "2",
  "hyperparameters": {
    "lr": "0.1",
    "units": "69"
  },
  "finalMetric": {
    "trainingStep": "19",
    "objectiveValue": 51.7642993461795
  },
  "startTime": "2021-02-26T02:12:03.812631906Z",
  "endTime": "2021-02-26T02:15:30Z",
  "state": "SUCCEEDED"
}
{
  "trialId": "1",
  "hyperparameters": {
    "lr": "0.01",
    "units": "232"
  },
  "finalMetric": {
    "trainingStep": "19",
    "objectiveValue": 55.58717541578339
  },
  "startTime": "2021-02-26T02:06:40.790962827Z",
  "endTime": "2021-02-26T02:10:47Z",
  "state": "SUCCEEDED"
}
{
  "trialId": "6",
  "hyperparameters": {
    "lr": "0.001",
    "units": "123"
  },
  "finalMetric": {
    "trainingStep": "19",
    "objectiveValue": 76.20946111911681
  },
  "startTime": "2021-02-26T02:31:11.163541810Z",
  "endTime": "2021-02-26T02:34:36Z",
  "state": "SUCCEEDED"
}
{
  "trialId": "4",
  "hyperparameters": {
    "lr": "0.01",
    "units": "246"
  },
  "finalMetric": {
    "trainingStep": "11",
    "objectiveValue": 100.75884600383479
  },
  "startTime": "2021-02-26T02:21:36.635969861Z",
  "endTime": "2021-02-26T02:25:02Z",
  "state": "SUCCEEDED"
}

ID None
Decay None
Learning Rate None
Validation Accuracy 0.0
```


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_bucket = True

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME